<a href="https://colab.research.google.com/github/CharlieChenyuZhang/ml_models/blob/master/kNN_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
Credit: https://github.com/Mujadded/knn-with-tensorflow/blob/master/k-nearest.py
"""

################################################################################
#########################Build a graph##########################################
################################################################################

#import libraries
from __future__ import print_function
import tensorflow as tf
import numpy as np
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

#importing the data
mnist=read_data_sets("data",one_hot=True)

#saving the datasets A.K.A Traning
# select 5000 images from MNIST as training data
# select 200 images from MNIST as test data
X_traning,Y_traning=mnist.train.next_batch(5000)
X_test,Y_test=mnist.test.next_batch(200)

#placeholders for variable to be used in model
#[None,28*28] because images in MNIST are 28x28 and we flaten it to a 2-D tensor
#[None,10] because of 10 categories from 0 - 9
xtr=tf.placeholder(tf.float32,[None,28*28]) #traning input
ytr=tf.placeholder(tf.float32,[None,10]) #traning label
xte=tf.placeholder(tf.float32,[28*28]) #testing input

#K-nearest
#NOTE: This is the hyperparameter you can change
K=10 #how many neighbors
nearest_neighbors=tf.Variable(tf.zeros([K]))

#model
#find the L1 distance
distance = tf.negative(tf.reduce_sum(tf.abs(tf.subtract(xtr, xte)),axis=1))
#tf.nn.top_k finds the k largest entries, tf.negative so that nn.top_k returns
# the result we want since we want to find k smallest
values,indices=tf.nn.top_k(distance,k=K,sorted=False)

#a normal list to save
nn = []
for i in range(K):
    nn.append(tf.argmax(ytr[indices[i]], 0)) #taking the result indexes

#saving list in tensor variable
nearest_neighbors=nn
#this will return the unique neighbors the count will return the most common's index
y, idx, count = tf.unique_with_counts(nearest_neighbors)

#pred is an integer representing the category of the digit
#suppose a is array([11,  1,  1,  1,  2,  2,  2,  3,  3,  4,  4,  4,  4,  4,  4,  4]) so unique_with_counts of a will
#return y= (array([ 1,  2,  3,  4, 11]) count= array([3, 3, 2, 7, 1])) so argmax of count will be 3 which will be the
#index of 4 in y which is the hight number in a
pred = tf.slice(y, begin=[tf.argmax(count, 0)], size=tf.constant([1], dtype=tf.int64))[0]

#initialize accuracy to 0
#initialize of all variables
accuracy=0
init=tf.global_variables_initializer()

################################################################################
##############################TF Session########################################
################################################################################

#start of tensor session
with tf.Session() as sess:
    for i in range(X_test.shape[0]):
        #return the predicted value
        #each X_test[i,:] is each test data, a 28 x 28 array
        #sess.run is the API to run the model and generate the prediction
        predicted_value=sess.run(pred,feed_dict={xtr:X_traning,ytr:Y_traning,xte:X_test[i,:]})

        # update accuracy
        if predicted_value == np.argmax(Y_test[i]):
            # if the prediction is right then a double value of 1./200 is added 200 here is the number of test
                accuracy += 1. / len(X_test)

#print out the accurascy
print(K,"-th neighbors' Accuracy is:",accuracy)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
10 -th neighbors' Accuracy is: 0.9350000000000007
